In [1]:
# country and variable mapping to make it easier to upload data
country_map = {
    "CH": "ch",
    "EU": "eu",
    "JP": "jp",
    "KR": "kr",
    "UK": "uk",
    "US": "us"
}

variable_map = {
    "TB3MS": "3m",
    "T10YR": "10y",
    "CPI": "cpi",
    "GDP": "gdp",
    "EXP": "exp",
    "IMP": "imp",
    "REC": "rec"
}


In [ ]:
# FUNCTIONALITY: Correctly parses dates in data
def smart_parse_dates(series):
  import pandas as pd

  # Excel serial date handling
  if pd.api.types.is_numeric_dtype(series):
    return pd.to_datetime(series, unit="D", origin="1899-12-30", errors="coerce")

  # Convert to string and strip spaces
  series = series.astype(str).str.strip()

  # Try common formats
  formats = [
    "%Y-%m-%d",
    "%Y-%m",
    "%Y/%m/%d",
    "%Y/%m",
    "%b %Y",
    "%Y"
  ]

  for fmt in formats:
    try:
      parsed = pd.to_datetime(series, format=fmt)
      # If at least some rows parsed, assume correct format
      if parsed.notna().sum() > 0:
        return parsed
    except:
      pass

  # Handle formats like "2024M01"
  try:
    series_mod = series.str.replace("M", "-", regex=False)
    parsed = pd.to_datetime(series_mod, format="%Y-%m")
    return parsed
  except:
    pass

  # Fallback
  return pd.to_datetime(series, errors="coerce")


In [ ]:
# FUNCTIONALITY: Loads data using specific path
from pathlib import Path
from typing import Union, Optional
import pandas as pd

def load_series_auto(path: Union[str, Path], date_col: Optional[str] = None):
  path = Path(path)
  folder = path.parent.name
  file = path.name

  # detect country
  ccode = file[:2].upper()
  if ccode not in country_map:
    raise ValueError(f"Unknown country code in filename: {file}")
  country = country_map[ccode]

  # detect variable folder
  if folder not in variable_map:
    raise ValueError(f"Unknown variable folder: {folder}")
  variable = variable_map[folder]

  col_name = f"{country}_{variable}"

  # load CSV
  if path.suffix == ".csv":
    # skip metadata lines
    with open(path) as f:
      lines = f.read().splitlines()
    header_index = next((i for i, line in enumerate(lines) if "DATE" in line or "date" in line), 0)
    df = pd.read_csv(path, skiprows=header_index)

  else:
    # handles multiple sheets
    xls = pd.ExcelFile(path)
    df = None

    for sheet in xls.sheet_names:
      temp = pd.read_excel(path, sheet_name=sheet)

      # find a date-like column
      for col in temp.columns:
        if temp[col].astype(str).str.match(r"\d{4}[-/]\d{2}").sum() > 3:
          df = temp
          date_col = col
          break

      if df is not None:
        break

    if df is None:
      # fallback: load first sheet
      df = pd.read_excel(path)
      date_col = df.columns[0]

  # ensure we have a date column
  if date_col is None:
    date_col = df.columns[0]

  # parse dates
  df[date_col] = smart_parse_dates(df[date_col])
  df = df.dropna(subset=[date_col])
  df = df.set_index(date_col).sort_index()

  # rename value column
  value_cols = [c for c in df.columns if c != date_col]
  df = df.rename(columns={value_cols[0]: col_name})

  return df, country, variable

In [ ]:
# FUNCTIONALITY: Imports all datasets in 'dataset' into all_sereies
from pathlib import Path

base = Path("dataset")
all_series = []

for folder in base.iterdir():        # e.g. dataset/3M, dataset/CPI
  if folder.is_dir():
    for file in folder.iterdir():    # <-- file declared here
      if file.suffix in [".csv", ".xlsx", ".xls"]:
        df, country, variable = load_series_auto(file)
        all_series.append(df)

In [ ]:
# FUNCTIONALITY: Turns all_series into a DataFrame
from functools import reduce

data = reduce(lambda left, right: left.join(right, how="outer"), all_series)
data = data.sort_index()


In [ ]:
print(data["us_gdp"].dropna())

observation_date
1947-01-01      243.164
1947-04-01      245.968
1947-07-01      249.585
1947-10-01      259.745
1948-01-01      265.742
                ...    
2024-04-01    29147.044
2024-07-01    29511.664
2024-10-01    29825.182
2025-01-01    30042.113
2025-04-01    30485.729
Name: us_gdp, Length: 314, dtype: float64
